###Load the Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/ML-Project/PII-DATA'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

# Load the CSV data into a DataFrame
df = pd.read_csv('/content/drive/My Drive/ML-Project/PII-DATA/token-labels.csv')

# Display the first few rows of the DataFrame to verify it's loaded correctly
df


,document,words,labels
0,7,"['Design', 'Thinking', 'for', 'innovation', 'r...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,10,"['Diego', 'Estrada', '\n\n', 'Design', 'Thinki...","['B-NAME_STUDENT', 'I-NAME_STUDENT', 'O', 'O',..."
2,16,"['Reporting', 'process', '\n\n', 'by', 'Gilber...","['O', 'O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME..."
3,20,"['Design', 'Thinking', 'for', 'Innovation', '\...","['O', 'O', 'O', 'O', 'O', 'B-NAME_STUDENT', 'I..."
4,56,"['Assignment', ':', '\xa0 ', 'Visualization', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...,...
11236,29429,"['Hello,', ""I'm"", 'Nicholas', 'Moore,', 'a', '...","['O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUDENT',..."
11237,29430,"['Hello,', 'my', 'name', 'is', 'Alexey', 'Novi...","['O', 'O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME..."
11238,29431,"['My', 'name', 'is', 'Ludmila', 'Inoue,', 'and...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD..."
11239,29432,"['Dr.', 'Tu', 'Garcia,', 'a', 'renowned', 'der...","['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'O',..."


In [3]:
unique_labels = [
   'O',
   'B-NAME_STUDENT',
    'I-NAME_STUDENT',
    'B-URL_PERSONAL',
    'B-EMAIL',
    'B-ID_NUM',
    'I-URL_PERSONAL',
    'B-USERNAME',
    'B-PHONE_NUM',
    'I-PHONE_NUM',
    'B-STREET_ADDRESS',
   'I-STREET_ADDRESS',
    'I-ID_NUM'
]

In [4]:
id2label = {
    0: 'O',
    1: 'B-NAME_STUDENT',
    2: 'I-NAME_STUDENT',
    3: 'B-URL_PERSONAL',
    4: 'B-EMAIL',
    5: 'B-ID_NUM',
    6: 'I-URL_PERSONAL',
    7: 'B-USERNAME',
    8: 'B-PHONE_NUM',
    9: 'I-PHONE_NUM',
    10: 'B-STREET_ADDRESS',
    11: 'I-STREET_ADDRESS',
    12: 'I-ID_NUM'
}

In [5]:
# import ast
# df['words'] = df['words'].apply(ast.literal_eval)
# df['labels'] = df['labels'].apply(ast.literal_eval)


###Text Preprocessing

In [6]:
import re
import ast

def clean_words_and_labels(words, labels):
    # Clean words and remove corresponding labels if the word is just whitespace or empty after cleaning.
    cleaned_words = []
    cleaned_labels = []
    for word, label in zip(words, labels):
        # Apply any specific cleaning necessary for each word
        cleaned_word = re.sub(r'\s+', '', word).strip()
        # Only add words and labels that are not empty after cleaning
        if cleaned_word:
            cleaned_words.append(cleaned_word)
            cleaned_labels.append(label)
    return cleaned_words, cleaned_labels

# Assuming 'words' and 'labels' columns contain string representations of lists
# Convert string representation of a list to actual list
df['words'] = df['words'].apply(ast.literal_eval)
df['labels'] = df['labels'].apply(ast.literal_eval)

# Apply the cleaning function to each pair of words and labels
cleaned_data = df.apply(lambda row: clean_words_and_labels(row['words'], row['labels']), axis=1)

# Separate the cleaned words and labels into their own columns
df['cleaned_words'] = cleaned_data.apply(lambda x: x[0])
df['cleaned_labels'] = cleaned_data.apply(lambda x: x[1])


In [7]:
df.drop(columns=['words', 'labels'], inplace=True)

In [8]:
df

,document,cleaned_words,cleaned_labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[Diego, Estrada, Design, Thinking, Assignment,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, by, Gilberto, Gamboa, Cha...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
3,20,"[Design, Thinking, for, Innovation, Sindy, Sam...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4,56,"[Assignment, :, Visualization, Reflection, Sub...","[O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_S..."
...,...,...,...
11236,29429,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
11237,29430,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
11238,29431,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
11239,29432,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [9]:
def remove_quotes_from_cleaned_words(df):
    # Removing quotes from each word in the 'cleaned_words' column
    df['cleaned_words'] = df['cleaned_words'].apply(lambda words: [word.replace('"', '') for word in words])
    return df


df = remove_quotes_from_cleaned_words(df)


In [10]:
def verify_and_display_alignment(df):
    # Check if each list of 'cleaned_words' has a corresponding list of 'cleaned_labels'
    for index, row in df.iterrows():
        words = row['cleaned_words']
        labels = row['cleaned_labels']
        if len(words) != len(labels):
            print(f"Mismatch found in document {index}: {len(words)} words, {len(labels)} labels")
            break
    else: # This else belongs to the for loop and executes if the loop doesn't break
        print("Each word has a corresponding label.")

    # Display words and labels for the first two 'sentences'
    for index, (words, labels) in enumerate(zip(df['cleaned_words'][2:5], df['cleaned_labels'][2:5])):
        print(f"\nSentence {index + 1}:")
        for word, label in zip(words, labels):
            print(f"{word}: {label}")

# Assuming 'df' is your dataframe and it contains 'cleaned_words' and 'cleaned_labels' columns
verify_and_display_alignment(df)


Each word has a corresponding label.

Sentence 1:
Reporting: O
process: O
by: O
Gilberto: B-NAME_STUDENT
Gamboa: I-NAME_STUDENT
Challenge: O
I: O
received: O
a: O
promotion: O
of: O
being: O
the: O
Regional: O
Controller: O
,: O
along: O
with: O
my: O
actual: O
position: O
of: O
Country: O
CFO: O
.: O
The: O
main: O
responsibility: O
of: O
this: O
new: O
position: O
was: O
to: O
weekly: O
report: O
the: O
results: O
for: O
the: O
week: O
and: O
estimate: O
the: O
final: O
results: O
of: O
the: O
month: O
of: O
4: O
countries: O
and: O
consolidated: O
those: O
.: O
When: O
I: O
was: O
receiving: O
the: O
position: O
,: O
I: O
went: O
to: O
visit: O
my: O
colleague: O
,: O
former: O
Regional: O
Controller: O
,: O
who: O
was: O
promoted: O
to: O
Country: O
CEO: O
and: O
now: O
had: O
interest: O
conflicts: O
of: O
being: O
the: O
controller: O
.: O
The: O
process: O
to: O
consolidate: O
the: O
information: O
of: O
the: O
4: O
countries: O
was: O
that: O
the: O
country: O
controllers: O
se

In [11]:
non_o_word_label_pairs = []

In [12]:
def print_non_O_words_and_labels(df):
    # Get the last five entries in the DataFrame
    last_five = df.iloc[100:105]

    # Iterate over each of the last five entries
    for index, (words, labels) in enumerate(zip(last_five['cleaned_words'], last_five['cleaned_labels'])):
        print(f"Document {index + len(df) - 5}:")
        # Create a list of tuples (word, label) where the label is not 'O'
        non_O_word_label_pairs = [(word, label) for word, label in zip(words, labels) if label != 'O']

        # Print each word with its non-'O' label
        for word, label in non_O_word_label_pairs:
            print(f"{word}: {label}")

# Call the function to print the words and their labels
# This should be run in your local environment
print_non_O_words_and_labels(df)


Document 11236:
Amal: B-NAME_STUDENT
Bhai: I-NAME_STUDENT
Document 11237:
Ahmed: B-NAME_STUDENT
Figo: I-NAME_STUDENT
Document 11238:
Guddu: B-NAME_STUDENT
Singh: I-NAME_STUDENT
Document 11239:
Ahmed: B-NAME_STUDENT
Salem: I-NAME_STUDENT
tps://www.facebook.com/bclark: B-URL_PERSONAL
https://www.youtube.com/channel/UC1ElAcppeuhfet: B-URL_PERSONAL
nYZqnhEXw: I-URL_PERSONAL
Ahmed: B-NAME_STUDENT
Salem: I-NAME_STUDENT
Document 11240:
Fabio: B-NAME_STUDENT
Sameh: I-NAME_STUDENT


In [13]:
id2label = {
    0: 'O',
    1: 'B-NAME_STUDENT',
    2: 'I-NAME_STUDENT',
    3: 'B-URL_PERSONAL',
    4: 'B-EMAIL',
    5: 'B-ID_NUM',
    6: 'I-URL_PERSONAL',
    7: 'B-USERNAME',
    8: 'B-PHONE_NUM',
    9: 'I-PHONE_NUM',
    10: 'B-STREET_ADDRESS',
    11: 'I-STREET_ADDRESS',
    12: 'I-ID_NUM'
}

# Invert the id2label mapping to create a label2id mapping
label2id = {label: id for id, label in id2label.items()}


# Corrected conversion function that can handle a list of lists
def convert_labels_to_integers(label_series, mapping):
    integer_labels = []
    for labels in label_series:
        int_label_row = []
        for label in labels:
            # Check if the label is in the mapping, otherwise ignore it
            if label in mapping:
                int_label_row.append(mapping[label])
        integer_labels.append(int_label_row)
    return integer_labels

# Apply the function to the 'cleaned_labels' column with the new mapping
df['integer_labels'] = convert_labels_to_integers(df['cleaned_labels'], label2id)

df[['cleaned_words', 'integer_labels']]


,cleaned_words,integer_labels
0,"[Design, Thinking, for, innovation, reflexion,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ..."
1,"[Diego, Estrada, Design, Thinking, Assignment,...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Reporting, process, by, Gilberto, Gamboa, Cha...","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[Design, Thinking, for, Innovation, Sindy, Sam...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[Assignment, :, Visualization, Reflection, Sub...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, ..."
...,...,...
11236,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11237,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11238,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11239,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
final = df[['cleaned_words', 'integer_labels']]

In [15]:
final.rename(columns={'integer_labels': 'cleaned_labels'}, inplace=True)

<ipython-input-15-c55765b59f3b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.rename(columns={'integer_labels': 'cleaned_labels'}, inplace=True)


In [16]:
final

,cleaned_words,cleaned_labels
0,"[Design, Thinking, for, innovation, reflexion,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ..."
1,"[Diego, Estrada, Design, Thinking, Assignment,...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Reporting, process, by, Gilberto, Gamboa, Cha...","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[Design, Thinking, for, Innovation, Sindy, Sam...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[Assignment, :, Visualization, Reflection, Sub...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, ..."
...,...,...
11236,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11237,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11238,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11239,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
from sklearn.model_selection import train_test_split

# Then you can use the train_test_split function
train_val_df, test_df = train_test_split(final, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=(1/9), random_state=42)


In [18]:
train_df

,cleaned_words,cleaned_labels
7245,"[During, my, tenure, as, a, designer, at, Matr...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9681,"[In, the, realm, of, healthcare,, I,, Delun, V...","[0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ..."
1097,"[STORYTELLING, IS, A, MUST, -, HAVE, TOOL, Cha...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5778,"[Reflection, :, Mind, Mapping, Tool, Describe,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3631,"[Reflection, -, Storytelling, Challenge, I, to...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
10145,"[In, the, realm, of, the, deceased,, where, th...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7254,"[As, a, jeweler,, I've, had, the, privilege, o...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
884,"[VISUALIZATION, Challange, and, selection, The...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5956,"[Reflection, -, Learning, Launching, Challenge...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
val_df

,cleaned_words,cleaned_labels
7714,"[In, the, annals, of, exemplary, nursing, prof...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ..."
9463,"[My, name, is, Kyle, Yamamoto,, and, I, work, ...","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6895,"[In, my, role, as, a, pharmacist, with, two, y...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
133,"[Mind, Mapping, Challenge, :, My, challenge, i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
404,"[Design, Thinking, for, Innovation, by, Univer...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, ..."
...,...,...
4220,"[Visualization, as, a, tool, Challenge, :, Fas...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9578,"[Greetings!, Allow, me, to, introduce, myself,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, ..."
5605,"[Learning, Launch, –, Communication, processes...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4424,"[Challenge, &, Selection, For, my, study, as, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [20]:
test_df

,cleaned_words,cleaned_labels
4170,"[Reflection, –, Mind, Mapping, Challenge, An, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6967,"[Hi,, my, name, is, Carmen, Chen, and, I, work...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
318,"[Federation, By, Sara, Garcia, Challenge, With...","[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3473,"[Example, Reflection, -, Learning, Launch, Cha...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7680,"[Isaac, Scott,, a, dedicated, school, teacher,...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
7234,"[In, a, bustling, city, where, skyscrapers, pi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5417,"[Mind, Mapping, Challenge, &, Selection, As, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4110,"[Reflection, Paper, on, Storytelling, Challeng...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7579,"[Hello,, my, name, is, Anil, Morozov., I, was,...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


###Tokenization

In [21]:


def insert_cls_sep(df):
    updated_data = []
    for _, row in df.iterrows():
        words = row['cleaned_words']
        labels = row['cleaned_labels']
        new_words = ['[CLS]']
        new_labels = [-101]

        for i, word in enumerate(words):
            # Append the word and its label
            new_words.append(word.strip('"'))  # Remove quotes and add word
            new_labels.append(labels[i])

            # Check if the word ends with a period or is a period
            if word.endswith('.'):
                # Append [SEP] after a word that ends with a period
                new_words.append('[SEP]')
                new_labels.append(-101)

                # If this word is not the last word, start a new sentence with [CLS]
                if i < len(words) - 1:
                    new_words.append('[CLS]')
                    new_labels.append(-101)

        # Ensure there is a [SEP] at the end of the last sentence if not already there
        if new_words[-1] != '[SEP]':
            new_words.append('[SEP]')
            new_labels.append(-101)

        updated_data.append({'cleaned_words': new_words, 'cleaned_labels': new_labels})

    return pd.DataFrame(updated_data)

# Assume train_df, val_df, and test_df are your DataFrames
# You would call the function like this:
updated_train_df = insert_cls_sep(train_df)
updated_val_df = insert_cls_sep(val_df)
updated_test_df = insert_cls_sep(test_df)




In [22]:
def check_cls_sep_tokens(df):
    first_row_words = df.iloc[0]['cleaned_words']
    first_row_labels = df.iloc[0]['cleaned_labels']
    assert len(first_row_words) == len(first_row_labels), "Mismatch in lengths of tokens and labels"

    for token, label in zip(first_row_words, first_row_labels):
        print(f"{token} ({label})", end=' ')

    print("\n\nCheck if [CLS] and [SEP] with -101 labels are correctly placed:")
    print("Start of sentence:", first_row_words[0], first_row_labels[0])
    print("End of sentence:", first_row_words[-1], first_row_labels[-1])

    # Check if [CLS] and [SEP] are correctly added after each period
    for i in range(1, len(first_row_words)-1):
        if first_row_words[i] == '.':
            print("Period detected at position", i)
            print("Should be [SEP] next:", first_row_words[i+1], first_row_labels[i+1])
            print("Should be [CLS] after [SEP] if not last:", first_row_words[i+2] if i+2 < len(first_row_words) else "N/A",
                  first_row_labels[i+2] if i+2 < len(first_row_labels) else "N/A")

# Use the function on your DataFrame
check_cls_sep_tokens(updated_train_df)


[CLS] (-101) During (0) my (0) tenure (0) as (0) a (0) designer (0) at (0) Matrix (0) Studios, (0) I (0) embarked (0) on (0) a (0) project (0) that (0) challenged (0) my (0) creativity (0) and (0) pushed (0) the (0) boundaries (1) of (0) design. (0) [SEP] (-101) [CLS] (-101) The (0) task (0) was (0) to (0) create (0) a (0) captivating (0) and (0) immersive (0) visual (0) experience (0) for (0) the (0) company's (0) latest (0) video (0) game, (0) Cosmic (0) Odyssey. (0) [SEP] (-101) [CLS] (-101) My (0) journey (0) began (0) with (0) extensive (0) research, (0) delving (0) into (0) the (0) realm (0) of (0) science (0) fiction (0) and (0) exploring (0) the (0) vastness (0) of (0) space. (0) [SEP] (-101) [CLS] (-101) I (0) spent (0) countless (0) hours (0) studying (0) celestial (0) bodies, (0) nebulae, (0) and (0) distant (0) galaxies, (0) drawing (0) inspiration (0) from (0) their (0) breathtaking (0) beauty. (0) [SEP] (-101) [CLS] (-101) Armed (0) with (0) this (0) knowledge, (0) I (0) 

In [23]:
def count_sentences_per_row(df):
    # Count the number of sentences based on [CLS] tokens
    df['sentence_count'] = df['cleaned_words'].apply(lambda words: words.count('[CLS]'))
    return df

#Call the function on the updated_train_df DataFrame
updated_train_df = count_sentences_per_row(updated_train_df)


#To see the sentence count for each row, you can now print or inspect the 'sentence_count' column:
print(updated_train_df['sentence_count'])


0       20
1       28
2       33
3       55
4       27
        ..
8987    29
8988    15
8989    51
8990    28
8991    15
Name: sentence_count, Length: 8992, dtype: int64


In [24]:
#Call the function on the updated_train_df DataFrame
updated_test_df = count_sentences_per_row(updated_test_df)


#To see the sentence count for each row, you can now print or inspect the 'sentence_count' column:
print(updated_test_df['sentence_count'])

0       26
1       13
2       23
3       15
4       21
        ..
1120    13
1121    13
1122    24
1123    16
1124    14
Name: sentence_count, Length: 1125, dtype: int64


In [25]:

#Call the function on the updated_train_df DataFrame
updated_val_df = count_sentences_per_row(updated_val_df)


#To see the sentence count for each row, you can now print or inspect the 'sentence_count' column:
print(updated_val_df['sentence_count'])

0       23
1       17
2       14
3       18
4       24
        ..
1119    29
1120    15
1121    13
1122    20
1123    15
Name: sentence_count, Length: 1124, dtype: int64


In [26]:

# Selecting only 'cleaned_words' and 'cleaned_labels' columns for train, test, and validation sets
train = updated_train_df[['cleaned_words', 'cleaned_labels']]
test = updated_test_df[['cleaned_words', 'cleaned_labels']]
val = updated_val_df[['cleaned_words', 'cleaned_labels']]


In [27]:
train

,cleaned_words,cleaned_labels
0,"[[CLS], During, my, tenure, as, a, designer, a...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[[CLS], In, the, realm, of, healthcare,, I,, D...","[-101, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, ..."
2,"[[CLS], STORYTELLING, IS, A, MUST, -, HAVE, TO...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[[CLS], Reflection, :, Mind, Mapping, Tool, De...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, -101, -101, ..."
4,"[[CLS], Reflection, -, Storytelling, Challenge...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
8987,"[[CLS], In, the, realm, of, the, deceased,, wh...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8988,"[[CLS], As, a, jeweler,, I've, had, the, privi...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8989,"[[CLS], VISUALIZATION, Challange, and, selecti...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8990,"[[CLS], Reflection, -, Learning, Launching, Ch...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [28]:
test

,cleaned_words,cleaned_labels
0,"[[CLS], Reflection, –, Mind, Mapping, Challeng...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[[CLS], Hi,, my, name, is, Carmen, Chen, and, ...","[-101, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[[CLS], Federation, By, Sara, Garcia, Challeng...","[-101, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[[CLS], Example, Reflection, -, Learning, Laun...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[[CLS], Isaac, Scott,, a, dedicated, school, t...","[-101, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
1120,"[[CLS], In, a, bustling, city, where, skyscrap...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1121,"[[CLS], Mind, Mapping, Challenge, &, Selection...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1122,"[[CLS], Reflection, Paper, on, Storytelling, C...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1123,"[[CLS], Hello,, my, name, is, Anil, Morozov., ...","[-101, 0, 0, 0, 0, 1, 2, -101, -101, 0, 0, 0, ..."


In [29]:
val

,cleaned_words,cleaned_labels
0,"[[CLS], In, the, annals, of, exemplary, nursin...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, ..."
1,"[[CLS], My, name, is, Kyle, Yamamoto,, and, I,...","[-101, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, -101, ..."
2,"[[CLS], In, my, role, as, a, pharmacist, with,...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[[CLS], Mind, Mapping, Challenge, :, My, chall...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[[CLS], Design, Thinking, for, Innovation, by,...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, ..."
...,...,...
1119,"[[CLS], Visualization, as, a, tool, Challenge,...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1120,"[[CLS], Greetings!, Allow, me, to, introduce, ...","[-101, 0, 0, 0, 0, 0, 0, 0, -101, -101, 0, 0, ..."
1121,"[[CLS], Learning, Launch, –, Communication, pr...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1122,"[[CLS], Challenge, &, Selection, For, my, stud...","[-101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [30]:
from transformers import BertTokenizerFast
import torch

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def sliding_window_tokenization_and_labels(words, labels, max_len=512, slide_len=256):
    tokenized_inputs = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    # Process each sequence with sliding windows
    start_index = 0
    while start_index < len(words):
        end_index = start_index + slide_len
        window_words = words[start_index:end_index]
        window_labels = labels[start_index:end_index]

        # Tokenization and padding to max length
        inputs = tokenizer(
            window_words,
            is_split_into_words=True,
            add_special_tokens=False,  # We already have [CLS] and [SEP] in our sequence
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Align labels with the tokenized word IDs, adding -101 for ignored tokens
        word_ids = inputs.word_ids(0)  # Batch index 0 since we're processing one sequence at a time
        window_aligned_labels = [-101 if word_id is None else window_labels[word_id] for word_id in word_ids]

        # Append the tokenized results
        tokenized_inputs['input_ids'].append(inputs['input_ids'].squeeze(0))
        tokenized_inputs['attention_mask'].append(inputs['attention_mask'].squeeze(0))
        tokenized_inputs['labels'].append(torch.tensor(window_aligned_labels, dtype=torch.long))

        # Move start index to the next slide
        start_index += slide_len

    # Stack all the tensors
    tokenized_inputs['input_ids'] = torch.stack(tokenized_inputs['input_ids'])
    tokenized_inputs['attention_mask'] = torch.stack(tokenized_inputs['attention_mask'])
    tokenized_inputs['labels'] = torch.stack(tokenized_inputs['labels'])

    return tokenized_inputs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [31]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Initialize lists to hold the tokenized data for all rows
train_input_ids = []
train_attention_masks = []
train_labels = []

# Example usage for each row in your DataFrame


for index, row in train.iterrows():
    tokenized_train_data = sliding_window_tokenization_and_labels(
        row['cleaned_words'], row['cleaned_labels']
    )
    train_input_ids.append(tokenized_train_data['input_ids'])
    train_attention_masks.append(tokenized_train_data['attention_mask'])
    train_labels.append(tokenized_train_data['labels'])


# Now concatenate the lists of tensors into single tensors
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.cat(train_labels, dim=0)

# Convert tokenized data into a TensorDataset
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)

# Define a batch size
batch_size = 16  # You can adjust this according to your requirements

# Create a DataLoader
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)


In [32]:
# Initialize lists to hold the tokenized data for all rows
test_input_ids = []
test_attention_masks = []
test_labels = []

# Tokenize all the data in the test set
for index,row in test.iterrows():
    tokenized_test_data = sliding_window_tokenization_and_labels(
        row['cleaned_words'], row['cleaned_labels']
    )
    test_input_ids.append(tokenized_test_data['input_ids'])
    test_attention_masks.append(tokenized_test_data['attention_mask'])
    test_labels.append(tokenized_test_data['labels'])


# Concatenate the lists of tensors into single tensors
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
test_labels = torch.cat(test_labels, dim=0)

# Convert the tokenized test data into a TensorDataset
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

# For test DataLoader, we usually don't need to shuffle the data, so we use the SequentialSampler
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)


In [33]:
# Initialize lists to hold the tokenized data for all rows
val_input_ids = []
val_attention_masks = []
val_labels = []

# Tokenize all the data in the validation set
for index, row in val.iterrows():
    tokenized_val_data = sliding_window_tokenization_and_labels(
        row['cleaned_words'], row['cleaned_labels']
    )
    val_input_ids.append(tokenized_val_data['input_ids'])
    val_attention_masks.append(tokenized_val_data['attention_mask'])
    val_labels.append(tokenized_val_data['labels'])

# Concatenate the lists of tensors into single tensors
val_input_ids = torch.cat(val_input_ids, dim=0)
val_attention_masks = torch.cat(val_attention_masks, dim=0)
val_labels = torch.cat(val_labels, dim=0)

# Convert the tokenized validation data into a TensorDataset
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

# For validation DataLoader, we usually don't need to shuffle the data either
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)


In [34]:
print(len(train_input_ids))
print(len(train_attention_masks))
print(len(train_labels))

print(len(test_input_ids))
print(len(test_attention_masks))
print(len(test_labels))

print(len(val_input_ids))
print(len(val_attention_masks))
print(len(val_labels))

24889
24889
24889
3090
3090
3090
3042
3042
3042


In [35]:
import torch
num_labels = len(id2label.values())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

label_counts = torch.zeros(num_labels)

# Count the occurrences of each label, excluding the special token index (-101)
for label_tensor in tokenized_train_data['labels']:
    labels, counts = label_tensor[label_tensor != -101].unique(return_counts=True)  # Make sure to use the correct ignore index here
    for label, count in zip(labels.tolist(), counts.tolist()):
        # Safeguard against any other unexpected label values
        if 0 <= label < num_labels:
            label_counts[label] += count

# Calculate the frequency of each label
label_freqs = label_counts.float() / label_counts.sum()
# Calculate weights as the inverse of the frequency
weights = 1.0 / (label_freqs + 1e-6)  # Add a small value to avoid division by zero

# Normalize the weights so that they sum up to the number of labels
weights = weights / weights.sum() * num_labels

# Convert to a tensor
label_weights = weights.to(device)


In [36]:
label_weights

tensor([2.3171e-06, 8.9864e-04, 2.9963e-04, 2.1663e+00, 8.9898e-05, 2.1663e+00,
        2.1663e+00, 2.1663e+00, 1.1237e-04, 2.1663e+00, 4.4942e-04, 2.9963e-04,
        2.1663e+00], device='cuda:0')

In [40]:
from transformers import BertForTokenClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
import torch.nn as nn
# Parameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_labels = len(id2label)
batch_size = 16
num_epochs = 50  # Higher number, since early stopping can halt training
learning_rate = 5e-5


model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
model.to(device)
model.train()


# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Loss function
loss_fn = nn.CrossEntropyLoss(weight=label_weights, ignore_index=-101)

# Early Stopping setup
patience = 3  # Number of epochs to wait after last time validation loss improved.
best_val_loss = float('inf')
best_accuracy = 0
no_improve_epochs = 0

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Other imports and setup code...
# Function to calculate accuracy per batch
def batch_accuracy(logits, labels):
    # Get the predictions and compare with true labels
    preds = torch.argmax(logits, dim=-1)
    mask = labels != -101  # Exclude the -101 labels from calculation
    corrects = (preds == labels) & mask  # Correct predictions
    accuracy = corrects.sum().item() / mask.sum().item()
    return accuracy

# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    total_accuracy = 0

    model.train()
    for batch in train_dataloader:
        input_ids, attention_masks, labels = batch
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks)

        loss = loss_fn(outputs.logits.view(-1, num_labels), labels.view(-1))
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        # Calculate batch accuracy from the logits and labels
        batch_acc = batch_accuracy(outputs.logits, labels)
        total_accuracy += batch_acc

    avg_train_loss = total_loss / len(train_dataloader)
    avg_train_accuracy = total_accuracy / len(train_dataloader)
    print(f'Epoch {epoch + 1}, Train Loss: {avg_train_loss}, Train Accuracy: {avg_train_accuracy}')

    # Validation phase
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_masks, labels = batch
            input_ids = input_ids.to(device)
            attention_masks = attention_masks.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_masks)

            loss = loss_fn(outputs.logits.view(-1, num_labels), labels.view(-1))
            val_loss += loss.item()

            # Calculate batch accuracy from the logits and labels
            batch_acc = batch_accuracy(outputs.logits, labels)
            val_accuracy += batch_acc

    avg_val_loss = val_loss / len(val_dataloader)
    avg_val_accuracy = val_accuracy / len(val_dataloader)
    print(f'Epoch {epoch + 1}, Validation Loss: {avg_val_loss}, Validation Accuracy: {avg_val_accuracy}')

    # Check if current epoch's validation loss is the best we've seen so far.
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_accuracy = avg_val_accuracy
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}.")
            print(f"Best Val Loss: {best_val_loss}, Best Val Accuracy: {best_accuracy}")
            break


Epoch 1, Train Loss: 0.18667618434033464, Train Accuracy: 0.9479307453411097
Epoch 1, Validation Loss: 0.08422747050797207, Validation Accuracy: 0.9748257011623371
Epoch 2, Train Loss: 0.1161441987717511, Train Accuracy: 0.9633235770446442
Epoch 2, Validation Loss: 0.10492002221704486, Validation Accuracy: 0.9498152069696445
Epoch 3, Train Loss: 0.05269826160721986, Train Accuracy: 0.9761773990400776
Epoch 3, Validation Loss: 0.039638195239575534, Validation Accuracy: 0.9823570228701259
Epoch 4, Train Loss: 0.08360709197173039, Train Accuracy: 0.9725030698822401
Epoch 4, Validation Loss: 0.12287675777537972, Validation Accuracy: 0.9567757570300566
Epoch 5, Train Loss: 0.055117726381615245, Train Accuracy: 0.9692135439507527
Epoch 5, Validation Loss: 0.03147548877424439, Validation Accuracy: 0.9708328558760622
Epoch 6, Train Loss: 0.042916190562108984, Train Accuracy: 0.9691393915152506
Epoch 6, Validation Loss: 0.039107803950764006, Validation Accuracy: 0.977204668037241


In [ ]:
# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader, RandomSampler, TensorDataset
# from transformers import BertForTokenClassification, AdamW, get_linear_schedule_with_warmup

# # Training loop
# for epoch in range(num_epochs):
#     total_loss = 0
#     model.train()
#     for batch in train_dataloader:
#         input_ids, attention_masks, labels = batch
#         input_ids = input_ids.to(device)
#         attention_masks = attention_masks.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(input_ids, attention_mask=attention_masks)
#         loss = loss_fn(outputs.logits.view(-1, num_labels), labels.view(-1))
#         loss.backward()
#         optimizer.step()
#         scheduler.step()

#         total_loss += loss.item()

#     avg_train_loss = total_loss / len(train_dataloader)
#     print(f'Epoch {epoch + 1}, Train Loss: {avg_train_loss}')

#     # Validation phase
#     model.eval()
#     val_loss = 0
#     with torch.no_grad():
#         for batch in val_dataloader:
#             input_ids, attention_masks, labels = batch
#             input_ids = input_ids.to(device)
#             attention_masks = attention_masks.to(device)
#             labels = labels.to(device)

#             outputs = model(input_ids, attention_mask=attention_masks)
#             loss = loss_fn(outputs.logits.view(-1, num_labels), labels.view(-1))
#             val_loss += loss.item()

#     avg_val_loss = val_loss / len(val_dataloader)
#     print(f'Epoch {epoch + 1}, Validation Loss: {avg_val_loss}')

#     # Early Stopping check
#     if avg_val_loss < best_val_loss:
#         best_val_loss = avg_val_loss
#         no_improve_epochs = 0
#     else:
#         no_improve_epochs += 1
#         if no_improve_epochs >= patience:
#             print("Early stopping triggered.")
#             break


In [ ]:
# import torch
# import torch.nn as nn
# from torch.optim import AdamW

# # Assuming `model` is already defined (BertForTokenClassification) and moved to the appropriate device
# model.to(device)
# model.train()
# num_epochs = 3
# # Define the optimizer


# optimizer = AdamW(model.parameters(), lr=5e-5)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * num_epochs)

# # Use a loss function that can utilize the label weights and ignore the special index
# loss_fn = nn.CrossEntropyLoss(weight=label_weights, ignore_index=-101)

# # Define the number of training epochs


# for epoch in range(num_epochs):
#     total_loss = 0
#     for batch in train_dataloader:
#         # Unpack the training batch from our dataloader.
#         input_ids, attention_masks, labels = batch
#         input_ids = input_ids.to(device)
#         attention_masks = attention_masks.to(device)
#         labels = labels.to(device)

#         # Clear any previously calculated gradients before performing a backward pass.
#         model.zero_grad()

#         # Forward pass (feed input data through the network)
#         outputs = model(input_ids, attention_mask=attention_masks)

#         # Compute loss, considering the label weights and ignore index
#         loss = loss_fn(outputs.logits.view(-1, model.num_labels), labels.view(-1))  # Flatten outputs and labels for loss calculation

#         # Backward pass to calculate the gradients
#         loss.backward()

#         # Update parameters and take a step using the computed gradient
#         optimizer.step()

#         # Accumulate the loss
#         total_loss += loss.item()

#     # Print average loss for the epoch
#     print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss/len(train_dataloader)}')


In [ ]:
# import torch
# import torch.nn as nn
# from torch.optim import AdamW



# # Use a loss function that can utilize the label weights and ignore the special index
# loss_fn = nn.CrossEntropyLoss(weight=label_weights, ignore_index=-101)

# # Define the number of training epochs
# num_epochs = 3

# for epoch in range(num_epochs):
#     total_loss = 0
#     for batch in train_dataloader:
#         # Unpack the training batch from our dataloader.
#         input_ids, attention_masks, labels = batch
#         input_ids = input_ids.to(device)
#         attention_masks = attention_masks.to(device)
#         labels = labels.to(device)

#         # Clear any previously calculated gradients before performing a backward pass.
#         model.zero_grad()

#         # Forward pass (feed input data through the network)
#         outputs = model(input_ids, attention_mask=attention_masks)

#         # Compute loss
#         loss = loss_fn(outputs.logits.view(-1, model.num_labels), labels.view(-1))  # Flatten outputs for loss calculation

#         # Backward pass to calculate the gradients
#         loss.backward()

#         # Update parameters and take a step using the computed gradient
#         optimizer.step()

#         # Accumulate the loss
#         total_loss += loss.item()

#     # Print average loss for the epoch
#     print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss/len(train_dataloader)}')
